In [1]:
import os
import time

import pandas as pd
import requests
from tqdm import tqdm

In [2]:
usernames = ["Fro116", "mpfei"]

In [3]:
os.chdir("../../data/user_profiles")

In [4]:
def get_score_df(json):
    # turn the json response into a dataframe
    plan_to_watch_status = 6
    dropped_status = 4
    on_hold_status = 3
    completed_status = 2
    currently_watching_status = 1

    volatile_rating_categories = [
        plan_to_watch_status,
        on_hold_status,
        currently_watching_status,
    ]

    anime_ids = []
    scores = []
    for item in response.json()["anime"]:
        if (
            item["watched_episodes"] == 1
            and item["total_episodes"] > 1
            and item["watching_status"] in volatile_rating_categories
        ):
            # preliminary ratings may be biased
            continue
        anime_ids.append(item["mal_id"])
        scores.append(item["score"])
    return pd.DataFrame({"anime_id": anime_ids, "my_score": scores})

In [5]:
user_lists = []

max_items_per_page = 300  # property of the API
for username in tqdm(usernames):
    more_pages = True
    page = 1
    scores = []
    while more_pages:
        time.sleep(5)
        response = requests.get(
            f"https://api.jikan.moe/v3/user/{username}/animelist/all/{page}"
        )
        page_df = get_score_df(response.json())
        scores.append(page_df)
        page += 1
        if len(page_df) < max_items_per_page:
            more_pages = False
    score_df = pd.concat(scores)
    score_df["username"] = username
    score_df = score_df.loc[lambda x: x["my_score"] != 0]
    user_lists.append(score_df)

100%|██████████| 2/2 [00:16<00:00,  8.27s/it]


In [6]:
scores = pd.concat(user_lists)

In [7]:
scores.to_pickle("ExtraUserAnimeLists.pkl")